# Noobai-XL-1.1 Text-to-Image Generator
### With advanced settings and HiRes Fix

This notebook allows you to use the Noobai-XL-1.1 model with customizable parameters.

In [ ]:
!pip install diffusers transformers torch accelerate xformers
!pip install opencv-python-headless
!pip install controlnet_aux

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler
from diffusers import DDIMScheduler, LMSDiscreteScheduler, KDPM2DiscreteScheduler
import cv2
import numpy as np
from PIL import Image
import json
import os

In [ ]:
# Function to modify tokenizer config to remove token limit
def modify_tokenizer_config(model_path):
    tokenizer_paths = [
        f"{model_path}/tokenizer/tokenizer_config.json",
        f"{model_path}/tokenizer_2/tokenizer_config.json"
    ]
    
    for path in tokenizer_paths:
        if os.path.exists(path):
            with open(path, 'r') as f:
                config = json.load(f)
            
            if 'model_max_length' in config:
                config['model_max_length'] = 2048  # Increased token limit
            
            with open(path, 'w') as f:
                json.dump(config, f, indent=2)

In [ ]:
# Load the model
model_id = "Laxhar/noobai-XL-1.1"
modify_tokenizer_config(model_id)

# Initialize pipeline with default scheduler
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None
)
pipe = pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
# Dictionary of available samplers
SAMPLERS = {
    'DPM++ 2M': DPMSolverMultistepScheduler,
    'Euler': EulerDiscreteScheduler,
    'Euler a': EulerAncestralDiscreteScheduler,
    'DDIM': DDIMScheduler,
    'LMS': LMSDiscreteScheduler,
    'KDPM2': KDPM2DiscreteScheduler
}

def change_sampler(pipe, sampler_name):
    if sampler_name in SAMPLERS:
        pipe.scheduler = SAMPLERS[sampler_name].from_config(pipe.scheduler.config)
    return pipe

# HiRes Fix upscaler function
def hires_fix_upscale(image, scale_factor=2, creativity_steps=20):
    # Convert PIL Image to numpy array
    img_np = np.array(image)
    
    # Upscale using NKMD-Superscale algorithm simulation
    upscaled = cv2.resize(img_np, None, fx=scale_factor, fy=scale_factor, 
                         interpolation=cv2.INTER_LANCZOS4)
    
    # Convert back to PIL Image
    return Image.fromarray(upscaled)

In [ ]:
def generate_image(prompt,
                   negative_prompt="",
                   width=512,
                   height=512,
                   steps=30,
                   cfg_scale=7.0,
                   sampler_name="DPM++ 2M",
                   seed=None,
                   use_hires_fix=False,
                   hires_scale=2,
                   hires_steps=20):
    
    # Set the seed if provided
    if seed is not None:
        generator = torch.Generator("cuda").manual_seed(seed)
    else:
        generator = torch.Generator("cuda").manual_seed(torch.randint(0, 2**32 - 1, (1,)).item())
    
    # Change sampler if needed
    pipe = change_sampler(pipe, sampler_name)
    
    # Generate the image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_inference_steps=steps,
        guidance_scale=cfg_scale,
        generator=generator
    ).images[0]
    
    # Apply HiRes Fix if requested
    if use_hires_fix:
        image = hires_fix_upscale(image, 
                                 scale_factor=hires_scale,
                                 creativity_steps=hires_steps)
    
    return image

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# Create interface widgets
prompt_text = widgets.Textarea(value='', description='Prompt:', rows=3)
negative_prompt = widgets.Textarea(value='', description='Negative:', rows=2)
width_slider = widgets.IntSlider(value=512, min=256, max=1024, step=64, description='Width:')
height_slider = widgets.IntSlider(value=512, min=256, max=1024, step=64, description='Height:')
steps_slider = widgets.IntSlider(value=30, min=10, max=150, description='Steps:')
cfg_slider = widgets.FloatSlider(value=7.0, min=1.0, max=20.0, step=0.5, description='CFG Scale:')
sampler_dropdown = widgets.Dropdown(options=list(SAMPLERS.keys()), description='Sampler:')
seed_text = widgets.Text(value='', description='Seed:', placeholder='Random if empty')

# HiRes Fix widgets
hires_checkbox = widgets.Checkbox(value=False, description='Use HiRes Fix')
hires_scale_slider = widgets.FloatSlider(value=2.0, min=1.5, max=8.0, step=0.5, description='HiRes Scale:')
hires_steps_slider = widgets.IntSlider(value=20, min=10, max=50, description='HiRes Steps:')

generate_button = widgets.Button(description='Generate')
output = widgets.Output()

def on_generate_button_click(b):
    with output:
        output.clear_output()
        
        # Convert seed to integer if provided
        seed = None if seed_text.value == '' else int(seed_text.value)
        
        image = generate_image(
            prompt=prompt_text.value,
            negative_prompt=negative_prompt.value,
            width=width_slider.value,
            height=height_slider.value,
            steps=steps_slider.value,
            cfg_scale=cfg_slider.value,
            sampler_name=sampler_dropdown.value,
            seed=seed,
            use_hires_fix=hires_checkbox.value,
            hires_scale=hires_scale_slider.value,
            hires_steps=hires_steps_slider.value
        )
        
        display(image)

generate_button.on_click(on_generate_button_click)

# Display interface
display(prompt_text,
        negative_prompt,
        widgets.HBox([width_slider, height_slider]),
        widgets.HBox([steps_slider, cfg_slider]),
        widgets.HBox([sampler_dropdown, seed_text]),
        hires_checkbox,
        widgets.HBox([hires_scale_slider, hires_steps_slider]),
        generate_button,
        output)